# Lab 3: Rag with Amazon SageMaker AI endpoint and Amazon OpenSearch and evaluate RAG with Ragas



## Overview
This notebook demonstrates how to implement a Retrieval Augmented Generation (RAG) solution using:
- Amazon SageMaker for hosting embedding and LLM models
- Amazon OpenSearch for vector search
- LangChain for orchestrating the RAG pipeline
- You'll explore ways to evaluate the quality of Retrieval-Augmented Generation (RAG) pipelines with the opensource tools like [RAGAS](https://docs.ragas.io/en/v0.1.21/index.html). You will use the OpenSearch Vector Database and the RAG results generation to show offline evaluation and scoring.

In this notebook, Question Answering solution with Large Language Models (LLMs) and Amazon OpenSearch Service. An application using the RAG(Retrieval Augmented Generation) approach retrieves information most relevant to the user’s request from the enterprise knowledge base or content, bundles it as context along with the user’s request as a prompt, and then sends it to the LLM to get a GenAI response.

LLMs have limitations around the maximum word count for the input prompt, therefore choosing the right passages among thousands or millions of documents in the enterprise, has a direct impact on the LLM’s accuracy.

<H2>Part 1: Build conversational search with OpenSearch Service</H2>

The vector dataset used in this part of the lab is comprised of a predefined content resource from the [PubMedQA](https://pubmedqa.github.io/) dataset.

You will use OpenSearch ingest pipeline with embedding processor to generate text embeddings for the dataset. Using the neural plugin in OpenSearch will allow you to generate the embeddings of the search query as well.
You will then use the large language model (LLM) hosted on Amazon SageMaker endpoints with the RAG processor in the search pipeline to generate text. The RAG processor will combine the retrieved search results from OpenSearch with the generated answer from the LLM to send back to the end user.

## 1.1. Import libraries & initialize resources
The code blocks below will install and import all the relevant libraries and modules used in this notebook.

In [ ]:
%pip uninstall -q -y autogluon-multimodal autogluon-timeseries autogluon-features autogluon-common autogluon-core

%pip install -Uq boto3==1.37.38
%pip install -Uq sagemaker==2.243.2
    
%pip install -Uq opensearch-py==2.8.0
%pip install -Uq opensearch_py_ml==1.1.0
%pip install -Uq deprecated==1.2.18
%pip install -Uq requests_aws4auth==1.3.1

%pip install -Uq transformers==4.51.3

%pip install -Uq datasets==3.5.0
%pip install -Uq ragas==0.2.14
%pip install -Uq python-dotenv==1.1.0

%pip install -Uq langchain==0.3.24
%pip install -Uq langchain-aws==0.2.21

%pip uninstall -yq packaging
%pip install -Uq packaging==24.1
    
print("Installs completed.")

In [ ]:
from IPython.display import display_html

display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

In [ ]:
# Import Python libraries
from typing import Any, Dict, List, Optional
import boto3
import json
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import os

import time

from transformers import AutoTokenizer

# Langchain
from langchain_core.messages import HumanMessage, SystemMessage

from langchain_community.embeddings import SagemakerEndpointEmbeddings
from langchain_community.embeddings.sagemaker_endpoint import EmbeddingsContentHandler
from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_community.llms import SagemakerEndpoint
from langchain_community.llms.sagemaker_endpoint import LLMContentHandler
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_aws.embeddings import BedrockEmbeddings


# Sagemaker
import sagemaker
from sagemaker.jumpstart.model import JumpStartModel
from sagemaker.huggingface import get_huggingface_llm_image_uri
from sagemaker.huggingface import HuggingFaceModel

# RAGAS
import ragas
from ragas.run_config import RunConfig
from ragas.metrics.base import MetricWithLLM, MetricWithEmbeddings
from ragas import evaluate
from ragas.metrics import Faithfulness, ResponseRelevancy
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.dataset_schema import SingleTurnSample

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [ ]:
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

region = sess.boto_region_name

account_id = boto3.client('sts').get_caller_identity().get('Account')

sm_runtime_client = boto3.client("sagemaker-runtime")
opensearch_client = boto3.client('opensearch')


print(f"account id: {account_id}")
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sagemaker_session_bucket}")
print(f"sagemaker session region: {region}")

## Load environment variables

In [ ]:
%store -r OS_DOMAIN_NAME
%store -r AOS_HOST
%store -r EMBEDDING_MODEL_NAME
%store -r EMBED_ENDPOINT_NAME
%store -r GENERATION_ENDPOINT_NAME

In [ ]:
print(f"OS_DOMAIN_NAME:{OS_DOMAIN_NAME}")
print(f"AOS_HOST:{AOS_HOST}")
print(f"EMBEDDING_MODEL_NAME:{EMBEDDING_MODEL_NAME}")
print(f"EMBED_ENDPOINT_NAME:{EMBED_ENDPOINT_NAME}")
print(f"GENERATION_ENDPOINT_NAME:{GENERATION_ENDPOINT_NAME}")

# 2. Build retrieval integration with OpenSearch

We have taken the PubMedQA dataset and prepared it to include the contexts in the `extracted_context.json` file.

The following cells will perform the steps to generate embeddings with the dataset and ingest into the OpenSearch vector database.

## 2.1 Establish a connection to the OpenSearch Service domain

### OpenSearch Configuration
- Establish connection to OpenSearch domain
- Create index with KNN vector search capabilities
- Define mapping for document embeddings

### 🚨 Authentication cell below 🚨 
The below cell establishes an authenticated connection to our OpenSearch Service domain. The connection will periodically expire.
If you see an `AuthorizationException` error later in this notebook it means that the connection has expired and you just need to re-run the cell to get a new security tokken.

In [ ]:
# Connect to OpenSearch using the IAM Role of this notebook
credentials = boto3.Session().get_credentials()
signerauth = AWSV4SignerAuth(credentials, region, "es")

# Create OpenSearch client
aos_client = OpenSearch(
    hosts=[f"https://{AOS_HOST}"],
    http_auth=signerauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=60
)
print("Connection details: ")
aos_client

## 2.2 Create the index with defined mappings.

It is important to define the 'knn_vector' fields as without the propper definitions dynamic mapping would type these as simple float fields.

A **k-NN (k-Nearest Neighbors)** enabled index is created in OpenSearch to store vector embeddings. The index schema defines:

A **knn_vector** field (`context_vector`) for storing embeddings.

**Note: ensure that the `dimension` parameter of your index mapping properties matches the dimensionality of your embedding model. In this example, `Alibaba-NLP/gte-base-en-v1.5` outputs vectors of `768` dimensions.**

To learn more about OpenSearch service, you can refer to the [document](https://aws.amazon.com/opensearch-service/).

In [ ]:
OPENSEARCH_INDEX_NAME = 'opensearch-rag-index'

In [ ]:
%store OPENSEARCH_INDEX_NAME

In [ ]:
### Create the k-NN index
# Check if the index exists. Delete and recreate if it does. 
if aos_client.indices.exists(index=OPENSEARCH_INDEX_NAME):
    print("The index exists. Deleting...")
    response = aos_client.indices.delete(index=OPENSEARCH_INDEX_NAME)
    
payload = { 
  "settings": {
    "index": {
      "knn": True
    }
  },
    "mappings": {
        "properties": {
            "context_vector": {
              "type": "knn_vector",
              "dimension": 768,
              "method": {
                "engine": "faiss",
                "space_type": "l2",
                "name": "hnsw",
                "parameters": {}
              }
            },
            "template": {
              "type": "keyword"
            }
          }
        }
}

print("Creating index...")
response = aos_client.indices.create(index=OPENSEARCH_INDEX_NAME,body=payload)
response

## 2.3 Use SageMaker Embedding Endpoint
In the prerequisites, you deployed a **text embedding model (Alibaba-NLP/gte-base-en-v1.5)** to a SageMaker real-time endpoint. This model converts text into 768-dimensional vectors for semantic search.

First, test the embeddings endpoint to ensure it is functional.

In [ ]:
query_text = "Is adjustment for reporting heterogeneity necessary in sleep disorders?"
# invoke the embedding model
input_str = {"inputs": query_text}
output = sm_runtime_client.invoke_endpoint(
    EndpointName=EMBED_ENDPOINT_NAME,
    Body=json.dumps(input_str),
    ContentType="application/json"
)
embeddings = output["Body"].read().decode("utf-8")
print(embeddings)

We can wrap up our SageMaker endpoints for embedding model into `langchain.embeddings.SagemakerEndpointEmbeddings` class to make it compatible with SageMaker embedding model and can be use with other LangChain functions.

In [ ]:
class EmbedContentHandler(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        """
        Transforms the input into bytes that can be consumed by SageMaker endpoint.
        Args:
            inputs: List of input strings.
            model_kwargs: Additional keyword arguments to be passed to the endpoint.
        Returns:
            The transformed bytes input.
        """
        # Example: inference.py expects a JSON string with a "inputs" key:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        """
        Transforms the bytes output from the endpoint into a list of embeddings.
        Args:
            output: The bytes output from SageMaker endpoint.
        Returns:
            The transformed output - list of embeddings
        Note:
            The length of the outer list is the number of input strings.
            The length of the inner lists is the embedding dimension.
        """
        # Example: inference.py returns a JSON string with the list of
        # embeddings in a "vectors" key:
        response_json = json.loads(output.read().decode("utf-8"))
        # print(len(response_json))
        return response_json


embed_content_handler = EmbedContentHandler()


embeddings_function = SagemakerEndpointEmbeddings(
    endpoint_name=EMBED_ENDPOINT_NAME,
    region_name=region,
    content_handler=embed_content_handler,
)

query_result = embeddings_function.embed_query(query_text)
print("Output:\n", query_result, end="\n\n")

## 2.4 Load data into the new index

### Data Processing
- Load and process input data
- Generate embeddings for documents
- Index documents with their embeddings in OpenSearch

We will use the [bulk API](https://opensearch.org/docs/latest/api-reference/document-apis/bulk/) to load all of the products into our newly created index. 

In [ ]:
def get_embedding(text, embed_endpoint_name, model_kwargs=None):
    """
    Call the SageMaker embedding model to embed the given text.
    Adjust the payload and response parsing according to your model's API.
    """
    embeddings = SagemakerEndpointEmbeddings(
        endpoint_name=embed_endpoint_name,
        region_name=region,
        content_handler=embed_content_handler,
    )

    return embeddings.embed_query(text)

get_embedding(query_text, EMBED_ENDPOINT_NAME)

- **Chunking**: Long documents are split into smaller passages (max 256 tokens) using LangChain's `RecursiveCharacterTextSplitter`.

- **Embedding Generation**: Each chunk is converted into a vector using the SageMaker embedding endpoint.

- **Bulk Ingestion**: The embeddings and text are indexed into OpenSearch for efficient retrieval.

In [ ]:
# Initialize tokenizer matching your embedding model (e.g., "sentence-transformers/all-mpnet-base-v2")
tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)

# Configure splitter with model-aware tokenization
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=250,  # 256 - safety buffer
    chunk_overlap=10,
    separators=["\n\n", "\n"],  # FIRST try splitting at paragraphs, then lines
    keep_separator=True,  # Preserve paragraph/line breaks in chunks
    is_separator_regex=False
)

def validate_chunk(chunk: str) -> bool:
    """Ensure chunk doesn't exceed token limit with model's actual tokenization"""
    tokens = tokenizer.encode(chunk, add_special_tokens=True)
    return len(tokens) <= 256

The below cell might take more than 10 mins, please be patient.

In [ ]:
%%time

input_filename = "extracted_context.json"
output_filename = "output_embedded.jsonl"  # Line-delimited JSON


# Load the input JSON file (mapping IDs to lists of context strings)
with open(input_filename, "r", encoding="utf-8") as infile:
    data = json.load(infile)

# Open the output file for writing line-delimited JSON objects
with open(output_filename, "w", encoding="utf-8") as outfile:
    for key, contexts in data.items():
        embeddings = []
        all_chunks = []
        for ctx_idx, context in enumerate(contexts):
                # First attempt: split at paragraphs/lines only
                chunks = text_splitter.split_text(context)

                # Second pass: check and fix any chunks that still exceed limits
                final_chunks = []
                for chunk in chunks:
                    if validate_chunk(chunk):
                        final_chunks.append(chunk)
                    else:
                        # Force split at sentences ONLY if absolutely necessary
                        emergency_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
                            tokenizer=tokenizer,
                            chunk_size=250,
                            chunk_overlap=50,
                            separators=[". "],  # Only split sentences when forced
                            keep_separator=True
                        )
                        final_chunks.extend(emergency_splitter.split_text(chunk))

                # Embed validated chunks
                for chunk_idx, chunk in enumerate(final_chunks):
                    if not validate_chunk(chunk):
                        continue  # Skip invalid chunks or handle differently

                    embedding = get_embedding(chunk, EMBED_ENDPOINT_NAME)
                    output_obj = {
                        "id": f"{key}-{ctx_idx}-{chunk_idx}",
                        "contexts": chunk,
                        "context_vector": embedding
                    }
                    outfile.write(json.dumps(output_obj) + "\n")

print(f"Embeddings saved to {output_filename}")

In [ ]:
# Read all JSON objects from the JSONL file
with open("output_embedded.jsonl", "r", encoding="utf-8") as infile:
    json_objects = [json.loads(line) for line in infile]

# Write the objects as a JSON array into a new .txt file
with open("merged_output.txt", "w", encoding="utf-8") as outfile:
    json.dump(json_objects, outfile, indent=4)

print("Merged JSON objects have been saved to merged_output.txt")

In [ ]:
def transform_file(input_filename, output_filename):
    # Load the merged file, which is expected to be a JSON array
    with open(input_filename, 'r', encoding='utf-8') as infile:
        records = json.load(infile)
    
    with open(output_filename, 'w', encoding='utf-8') as outfile:
        # Process each record in the array
        for record in records:
            contexts = record.get("contexts", [])
            vectors = record.get("context_vector", [])
            # For each pair of context string and corresponding embedding vector:
            # Create a new object without the "id" field.
            new_obj = {
                "contexts": contexts,
                "context_vector": vectors
            }
            # Write the JSON object as a single line
            outfile.write(json.dumps(new_obj) + "\n")

transform_file("merged_output.txt", "final_output_oneline.txt")
print("Transformation complete. Check final_output.txt")


In [ ]:
## Index TEXT file into index: opensearch-rag-index
batch = 0
count = 0
batch_size = 5
body_ = ''
action = json.dumps({ 'index': { '_index': OPENSEARCH_INDEX_NAME } })
errors = []
with open('final_output_oneline.txt', 'r') as file:
    for line in file:
        if count > 5000:
            break # Use this to run a limited number of items.
        body_ = body_ + action + "\n" + line + "\n"
        # print(f"body: {body_}")
        if count % batch_size == 0 and count != 0:
            batch+=1
            if count % (batch_size*30) == 0:
                print("Batch: " + str(batch) + ", count: " + str(count)+ ", errors: " + str(len(errors)))
            response = aos_client.bulk(
                index = OPENSEARCH_INDEX_NAME,
                body = body_
            )
            body_ = ''
            if response['errors'] == True:
                for item in response['items']:
                    if item['index']['status'] != 201:
                        errors.append(item['index']['error']) 
        # print(response)
        # break 
        count += 1
if body_ !="":
    response = aos_client.bulk(
        index = OPENSEARCH_INDEX_NAME,
        body = body_
    )
if response['errors'] == True:
    for item in response['items']:
        if item['index']['status'] != 201:
            errors.append(item['index']['error'])
print("Last batch: " + str(batch) + ", document count: " + str(count)+ ", errors: " + str(len(errors)))

## 2.5 Query OpenSearch Database

Once the vectors are ingested into the database, we can run queries to retrieve relevant contexts based on the input query.

In [ ]:
# Your natural language query
query_vector = get_embedding(query_text, EMBED_ENDPOINT_NAME)

# Now, use the embedding in a k-NN query
knn_query = {
    "size": 5,  # adjust how many results you want to retrieve
    "query": {
        "knn": {
            "context_vector": {
                "vector": query_vector,
                "k": 5
            }
        }
    }
}

response_knn = aos_client.search(
    index=OPENSEARCH_INDEX_NAME,
    body=knn_query
)

print("KNN Query Results:")
for hit in response_knn['hits']['hits']:
    print(hit['_source'])



# 3. Build end-to-end RAG pipeline with LLM models hosted on SageMaker AI and LangChain

We plan to use document embeddings to fetch the most relevant documents in our document knowledge library and combine them with the prompt that we provide to LLM.

To achieve that, we will do following.

1. **Generate embedings for each of document in the knowledge library with SageMaker hosted embedding model.**
2. **Identify top K most relevant documents based on user query.**
    - 2.1 **For a query of your interest, generate the embedding of the query using the same embedding model.**
    - 2.2 **Search the indexes of top K most relevant documents in the embedding space using in-memory Faiss search.**
    - 2.3 **Use the indexes to retrieve the corresponded documents.**
3. **Combine the retrieved documents with prompt and question and send them into SageMaker LLM.**



Note: The retrieved document/text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM prompt -- maximum sequence length of 1024 tokens. 

---
To build a simiplied QA application with LangChain, we need: 
1. Wrap up our SageMaker endpoints for embedding model and LLM into `langchain.embeddings.SagemakerEndpointEmbeddings` and `langchain.llms.sagemaker_endpoint.SagemakerEndpoint`. (We have already created the embedding SageMaker wrapper class in the previous section.
2. Prepare the dataset to build the knowledge data base. 

---

Now you will use the `Llama 3.1 8B` model you deployed to a SageMaker real-time endpoint in the prerequisites with LangChain.

Invoke the LLM endpoint for a quick test:

In [ ]:
input_str = { "inputs": query_text, 
            "parameters": { 
                "max_new_tokens": 100, 
                "top_p": 0.9, 
                "temperature": 0.6 
            }
        }
output = sm_runtime_client.invoke_endpoint(
    EndpointName=GENERATION_ENDPOINT_NAME,
    Body=json.dumps(input_str),
    ContentType="application/json"
)
embeddings = output["Body"].read().decode("utf-8")
print(embeddings)

Next, we wrap up our SageMaker endpoints for LLM into `langchain.llms.sagemaker_endpoint.SagemakerEndpoint`. 

In [ ]:
parameters = {
    "max_new_tokens": 100,
    "temperature": 0.2,
    "top_p": 0.9
}


class GenerationContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        self.len_prompt = len(prompt)
        input_str = json.dumps({"inputs": prompt, "parameters": {**model_kwargs}})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = output.read()
        res = json.loads(response_json)
        
        ans = res['generated_text']
        # print(ans)
        return ans

In [ ]:
generation_content_handler = GenerationContentHandler()

sm_llm = SagemakerEndpoint(
    endpoint_name=GENERATION_ENDPOINT_NAME,
    region_name=region,
    model_kwargs=parameters,
    content_handler=generation_content_handler,
)

We combine the retrieved documents with prompt and question and send them into SageMaker LLM.

We define a customized prompt as below.

In [ ]:
from langchain import PromptTemplate

prompt_template = """
        <|begin_of_text|>
        <|start_header_id|>system<|end_header_id|>
        You are an assistant for question-answering tasks. Answer the following question using the provided context. If you don't know the answer, just say "I don't know.".
        <|start_header_id|>user<|end_header_id|>
        Context: {context}
        
        Question: {question}
        <|start_header_id|>assistant<|end_header_id|> 
        Answer:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

opensearch_url = f"https://{AOS_HOST}"


For this example, we have created the OpenSearch cluster using an IAM role as the master user (your SageMaker execution role). In real world application, you will be authenticating with other user accounts, and should store the user name and password using services that can securely store the value, for example SecretsManager as [shown here](https://github.com/aws-samples/rag-with-amazon-opensearch-and-sagemaker/blob/main/app/opensearch_retriever_llama2.py#L89).

In [ ]:
opensearch_vector_search = OpenSearchVectorSearch(
    opensearch_url=opensearch_url,
    index_name=OPENSEARCH_INDEX_NAME,
    embedding_function=embeddings_function,
    http_auth=signerauth,
    connection_class=RequestsHttpConnection
)

In [ ]:
retriever = opensearch_vector_search.as_retriever(
    search_kwargs={"k": 3, "vector_field": "context_vector", "text_field": "contexts"})


In [ ]:
chain_type_kwargs = {"prompt": PROMPT, "verbose": True}
qa = RetrievalQA.from_chain_type(
    sm_llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    # return_source_documents=True, ## you can uncomment this line to see the detailed retrieved data source
    # verbose=True, #DEBUG
)


In [ ]:
qa(query_text)

# 📂 Test Evaluation Pipeline


### 📊 RAGAS Evaluation Metrics

We're going to measure the following aspects of a RAG system. These metrics are defined in **[RAGAS](https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/)**:

- 🔍 **[Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/faithfulness/)**  
  Measures how factually consistent the generated answer is with the retrieved context. It evaluates whether the answer could reasonably be derived from the context.

- 🎯 **[Response Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/answer_relevance/)**  
  Assesses how relevant the generated answer is to the original user query. A high score indicates the answer is on-topic and useful.

- 🧠 **[Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/context_precision/)**  
  Measures how many of the retrieved contexts are truly relevant to answering the question. Precision reflects the "purity" of the retrieved chunks.

- 📥 **[Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/context_recall/)**  
  Evaluates how well the retrieved context covers the information needed to answer the question completely. High recall means fewer relevant facts are missed.

- 🧬 **[Answer Similarity](https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/answer_similarity/)**  
  Compares the generated answer to a reference answer (if available), measuring how semantically close they are using embedding-based similarity.

- ✅ **[Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/answer_correctness/)**  
  Evaluates whether the generated answer is factually correct and aligns with known ground-truth answers, if such references are available.

> 📚 Want to dive deeper into how each metric is computed?  
Check out the full [RAGAS metrics documentation](https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/).


In [ ]:
# import metrics
metrics=[
        ragas.metrics.answer_relevancy,
        ragas.metrics.faithfulness,
        ragas.metrics.context_precision,
        ragas.metrics.context_recall,
        # ragas.metrics.answer_similarity, # uncomment to add this metric
        # ragas.metrics.answer_correctness, # uncomment to add this metric
    ]

In [ ]:
# util function to init Ragas Metrics
def init_ragas_metrics(metrics, llm, embedding):
    for metric in metrics:
        if isinstance(metric, MetricWithLLM):
            print(metric.name + " llm")
            metric.llm = llm
        if isinstance(metric, MetricWithEmbeddings):
            print(metric.name + " embedding")
            metric.embeddings = embedding
        run_config = RunConfig()
        metric.init(run_config)

Because some RAGAS metrics contain both [LLM-based and non-LLM-based](https://docs.ragas.io/en/stable/concepts/metrics/overview/#different-types-of-metrics), we will provide the necessary LLM and embedding models to do the evaluation. Here we will use the LLM and embedding model provided by Amazon bedrock to do the evaluation.

In [ ]:
from langchain_aws import ChatBedrock as LangChainBedrock
# Use correct region for Titan Embed (e.g., us-east-1)
client = boto3.client(
    "bedrock-runtime",
    region_name=region,  # Titan Embed supported here
)

emb = BedrockEmbeddings(
    client=client,
    model_id="amazon.titan-embed-text-v1",  # Case-sensitive!
)
init_ragas_metrics(
    metrics,
    llm=LangchainLLMWrapper(LangChainBedrock(model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0")),
    embedding=LangchainEmbeddingsWrapper(emb),
)

Now we have to initialize the metrics with LLMs and embedding models of your choice. In this example we are going to use the claude-3-5-haiku model and amazon.titan-embed-text-v1 embedding model, and use the convenience wrappers from the `langchain-aws` library.

### Creating the Sagemaker Endpoint for Llama 3.1 8b Instruct Model

### Score with RAGAS

In this example, we will demonstrate both solutions using prebuilt dataset and a live RAG pipeline with AWS Open Search and evaluate the results using RAGAS.

In [ ]:
async def score_with_ragas(query, chunks, answer, metrics, reference):
    scores = {}
    for metric in metrics:
        sample = SingleTurnSample(
            user_input=query,
            retrieved_contexts=chunks,
            response=answer,
            reference=reference
        )
        print(f"calculating {metric.name}")
        scores[metric.name] = await metric.single_turn_ascore(sample)
    return scores

#### Scoring RAG
We have already setup the Open Search Database in the first section, we can now **evaluate** the quality of its results against a test dataset - to help us **optimize** the configuration for high quality and low cost.

First, let's load the sample dataset of questions, reference answers, and their source documents (to find more of how to prepare this dataset, please see more details in [this github](https://github.com/aws-samples/llm-evaluation-methodology/blob/main/datasets/Prepare-SQuAD.ipynb)):


In [ ]:
import pandas as pd
dataset_df = pd.read_csv("ori_pqal_10_records.csv")
sample_df = dataset_df.head(1)
sample_df

Records in this dataset include:

- (`CONTEXT`) The full text of the source document for this example
- (`QUESTION`) The user question to be asked
- (`LONG_ANSWER`) A reference 'correct' answer, supported by the document

As shown in [Ragas' API Reference](https://docs.ragas.io/en/latest/references/evaluation.html), records in Ragas evaluation datasets typically include:

- The `question` that was asked
- The `answer` the system generated
- The actual text `contexts` the answer was based on (i.e. snippets of document text retrieved by the search engine)
- The `ground_truth` answer(s)


We can run an example question through the OpenSearch Vector database to retrieve and generate pipeline as shown below, and extract the references ready to calculate metrics.

In [ ]:

def retrieve_and_generate(
    question: str,
    top_k: int = 3,
    system_prompt: str = "You are a helpful assistant. Use the context to answer concisely.",
    **kwargs,
):
    # Step 1: Retrieve relevant context from OpenSearch
    # Your natural language query
    query_vector = get_embedding(question, EMBED_ENDPOINT_NAME)

    # Now, use the embedding in a k-NN query
    knn_query = {
        "query": {
            "knn": {
                "context_vector": {
                    "vector": query_vector,
                    "k": top_k
                }
            }
        }
    }
    response = aos_client.search(
        index=OPENSEARCH_INDEX_NAME,
        body=knn_query
    )
    
    hits = response["hits"]["hits"]
    contexts = [hit["_source"]["contexts"] for hit in hits]
    doc_ids = [hit["_id"] for hit in hits]

    # Step 2: Format prompt with retrieved context
    combined_context = "\n\n".join(contexts)
    full_prompt = f"""Context:
            {combined_context}
            
            Question: {question}
            Answer:
            """

    # Step 3: Call your SageMaker-hosted model using LangChain
    messages: List = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=full_prompt)
    ]
    
    response_chunk = sm_llm.invoke(messages)
    answer = response_chunk  # already joined by your handler


    return {
        "answer": answer,
        "retrieved_doc_ids": doc_ids,
        "retrieved_doc_texts": contexts
    }

Run RAG as requests come in and score the results immediately.

In [ ]:
from asyncio import run

def rag_pipeline(
    question: str,
    reference: str,
    metrics: Optional[Any] = None,

):
    generated_answer = retrieve_and_generate(
        question=question,
        top_k=3,  # or whatever makes sense for your context window
        system_prompt="You are a helpful assistant. Use the context below to answer the question."
    )

    answer = generated_answer["answer"]
    contexts = generated_answer["retrieved_doc_texts"]
    print(f"question: {question}")
    print(f"answer: {generated_answer["answer"]}")
    print(f"reference: {reference}")

    score = run(score_with_ragas(question, contexts, answer=answer, metrics=metrics, reference=reference))

    return score



In [ ]:
%%time
scores = []
for index, row in sample_df.iterrows():
    response = rag_pipeline(
        question=row["QUESTION"],
        reference=row["LONG_ANSWER"],
        metrics=metrics
    )
    scores.append(response)
    print(f"\n\nscore: {json.dumps(response, indent=4)}\n\n")

### Key Workflow Summary
- Data Preparation: Text is split into chunks and embedded.
- OpenSearch Setup: A vector index is created and populated.
- Model Deployment: Embedding and LLM models are hosted on SageMaker.
- RAG Pipeline: Queries retrieve relevant context, and the LLM generates answers.


This notebook provides an end-to-end example of building a production-ready RAG system using AWS services. The same approach can be adapted for other domains by replacing the dataset and fine-tuning the models.

# Congratulations for finishing Lab 3. Now please continue on to the next Lab.